# Global Impact of the 2008 Recession
By: Andre Soghomonians

## Table of Contents
#### [1. Introduction](#intro)
#### [2. Pulling the Data](#pulling_data)
#### [3. Economic Data](#economic_data)
#### [4. Crime Data](#crime_data)
#### [5. Conclusion](#conclusion)

## 1. Introduction <a id="intro"></a>

This project will look at how the world was impacted by the 2008 recession with the two primary focuses will be on the economy and crime. Each country's GDP growth rate will serve as a proxy for their economic condition and intentional homicide rate per 100,000 will be used as a measure for crime rate. The scope of the analysis will be both on a regional and national level. The data comes from the dataset ["World Development Indicators"](https://www.kaggle.com/datasets/psycon/world-development-indicators) uploaded by Kaggle user Batucan Senkal. It contains a multitude of economic indicators from every country starting in 1960.  

## 2. Pulling the Data <a id="pulling_data"></a>

In [ ]:
# Libraries 
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy import stats

In [ ]:
# Connecting to SQL
db_path = '/kaggle/input/world-development-indicators/indicators.sqlite'
conn = sql.connect(db_path)
print('Connected')

In [ ]:
# Listing all tables in dataset 
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)
print(table)

In [ ]:
# Indicator table
query = 'SELECT * FROM indicators;'
indicators = pd.read_sql_query(query, conn)
indicators.head(10)

In [ ]:
# Country table
query = 'SELECT * FROM Country;'
country_info = pd.read_sql_query(query, conn)
country_info.head(10)

## 3. Economic Data <a id="economic_data"></a>

I decided to use GDP growth rate to measure each of their economies because it was one of the most available variables in the dataset. If I were to use raw GDP instead, the values would have been much larger which would make it more difficult to interpret. Although GDP growth rate does not capture every aspect of a nation's economy, there has been extensive research that has shown it is correlated with other important indicators that measure quality of life such as literacy rate and infant mortality rate among others. 

In [ ]:
# Creating table of GDP values by region
query = """
        SELECT c.Region, i.Year, AVG(i.Value) AS GDP_Growth
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "NY.GDP.MKTP.KD.ZG" AND c.Region != ''
        GROUP BY i.Year, c.Region
        """
gdp_data = pd.read_sql_query(query, conn)
gdp_data.head(10)

In [ ]:
# Giving each region its own column
gdp = pd.pivot(gdp_data, index = 'Year', columns = 'Region', values = 'GDP_Growth').reset_index()
gdp.head()

In [ ]:
# Summary statistics of each region
gdp.loc[:, gdp.columns != "Year"].describe()

From the summary statistics of the entire dataset, we can see that the Middle East & North Africa has been the most volatile in terms of GDP growth. Although they have the highest mean growth, their standard deviation is by far the highest at 6.433%. The next closest is Europe & Central Asia at 2.576%. The Middle East & North Africa also have the biggest range, with their maximum growth being 44.893% and lowest being 0.231%.

In [ ]:
# Color codes for regions
region_col = {'East Asia & Pacific': 'firebrick', 'Europe & Central Asia': 'darkkhaki', 
           'Latin America & Caribbean': 'royalblue', 'Middle East & North Africa': 'darkorange',
           'North America': 'forestgreen', 'South Asia': 'darkmagenta', 'Sub-Saharan Africa': 'sienna'}

# Color codes for countries
country_col = {}
for index, row in country_info.iterrows():
    if row['Region'] == 'East Asia & Pacific':
        country_col[row['TableName']] = 'firebrick'
    elif row['Region'] == 'Europe & Central Asia':
        country_col[row['TableName']] = 'darkkhaki'
    elif row['Region'] == 'Latin America & Caribbean':
        country_col[row['TableName']] = 'royalblue'
    elif row['Region'] == 'Middle East & North Africa':
        country_col[row['TableName']] = 'darkorange'
    elif row['Region'] == 'North America':
        country_col[row['TableName']] = 'forestgreen'
    elif row['Region'] == 'South Asia':
        country_col[row['TableName']] = 'darkmagenta'
    elif row['Region'] == 'Sub-Saharan Africa':
        country_col[row['TableName']] = 'sienna'

In [ ]:
# Visualizing differences by region

for i, j in region_col.items():
    plt.plot(gdp['Year'], gdp[i], color = j, label = i)
plt.xlabel('Year')
plt.ylabel('GDP Growth')
plt.title('GDP Growth by Region (2005-2010)')
plt.xlim(2005,2010)
plt.ylim(-10,10)
plt.legend(loc='lower left')
plot1 = plt.show()

As expected, each region saw a decrease in growth during the recession. However, only three saw their growth fall below 0. Another interesting fact is the most volatile region during the entire dataset, the Middle East & North Africa, saw little change in comparison to other regions. 

In [ ]:
# Pulling each country's lowest year of GDP growth
query = """
        SELECT i.CountryName, c.CountryCode, i.Year, MIN(i.Value) AS MinGrowth
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "NY.GDP.MKTP.KD.ZG" AND c.Region != ''
        GROUP BY i.CountryName
        """
min_growth = pd.read_sql_query(query, conn)
min_growth.head(10)

In [ ]:
# Calculating the proportion of countries who suffered their lowest growth during the recession. 
recession_impact = min_growth[(min_growth['Year'] > 2006) & (min_growth['Year'] < 2010)].sort_values(by = 'MinGrowth')
prop = round((len(recession_impact.index)/len(min_growth.index)) * 100, 2)
print('Proportion of Countries Hit Hardest by Recession:', prop, '%')
recession_impact.head(10)

In all countries in the dataset, 20.49% of them experienced their lowest GDP growth during the recession. Given the dataset spans all the way back to 1960, I find this number to be somewhat surprising. That time span includes siginificant historical events such as decolonization, multiple wars, the fall of the Soviet Union, and many more. I would have expected the proportion to be slightly lower. 

In [ ]:
# Top 10 Countries hit hardest by Recession
query = """
        SELECT i.CountryName, c.CountryCode, i.Year, MIN(i.Value) AS MinGrowth
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "NY.GDP.MKTP.KD.ZG" AND c.Region != '' AND i.Year BETWEEN 2007 AND 2009
        GROUP BY i.CountryName
        ORDER BY MinGrowth
        LIMIT 10"""
top_10 = pd.read_sql_query(query, conn)
top_10.head(10)

In [ ]:
# Graph of the top 10 countries impacted the hardest

# Pulling necessary colors
colors = {}
for index, row in top_10.iterrows():
    if top_10['CountryName'][index] in country_col: 
        colors[row['CountryName']] = country_col.get(row['CountryName'])

# Creating the graph
plot = plt.barh(y = 'CountryName', width = 'MinGrowth', data = top_10, color = [colors.get(i) for i in top_10['CountryName']])    
plt.bar_label(plot, label_type = 'center')
plt.xlim(-20,0)
plt.title('Top 10 Biggest Percentage Decreases in GDP (2007-2009)')

# yticks
top_10['Country and Year'] = top_10['CountryName'] + " " + top_10['Year'].astype(str)
plt.yticks(range(len(top_10)), top_10['Country and Year'])

# Creating the legend
titles = []
for region, color in region_col.items():
    titles.append(mpatches.Patch(color=color, label=region))
titles = titles[:4] + titles[6:]
plt.legend(handles = titles, bbox_to_anchor = (1.25,0.5), loc = 'center')

plt.show()

When focusing the scope to the national level, it becomes apparent how catastrophic the effect was for specific countries. Zimbabwe suffered greatly with a 17.67% decrease in their GDP in 2008. The next five with the lowest GDP growth are all countries in eastern Europe, while the rest were in other parts of the world. When comparing this to the regional level, it is surprising to see none from North America since it suffered around the same as Europe & Central Asia. 

## 4. Crime Data <a id="crime_data"></a>

Intentional homicide rate was selected because there is empirical evidence connecting crime rate to the economic status of a country. It is also one of the few crime statistics available in the dataset. 

In [ ]:
# Pulling regional homicide rate data
query = """
        SELECT c.Region, i.Year, AVG(i.Value) AS HomRate
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "VC.IHR.PSRC.P5" AND c.Region != ''
        GROUP BY i.Year, c.Region
        """
hom_data = pd.read_sql_query(query, conn)
hom_data.head(10)

In [ ]:
# Giving each region its own column
hom = pd.pivot(hom_data, index = 'Year', columns = 'Region', values = 'HomRate').reset_index()
hom.head()

In [ ]:
# Statistical summary of each region
hom.loc[:, hom.columns != "Year"].describe()

Unsurprisingly, Latin America & Caribbean has by far the highest average intentional homicide rate. Countries such as Mexico, Brazil, and Colombia are infamous for how rampant organized crime has been for them. Sub-Saharan Africa also has a high average and is even more volatile than Latin America & Caribbean. Their standard deviation of 7.71 is by far the highest, demonstrating the effects of  overall instability within the region. 

In [ ]:
# Graph of homicide rate
for i, j in region_col.items():
    plt.plot(hom['Year'], hom[i], color = j, label = i)
plt.xlabel('Year')
plt.ylabel('Intentional Homicide Rate (Per 100,000)')
plt.title('Intentional Homicide Rate by Region (2005-2010)')
plt.xlim(2005,2010)
plt.ylim(0, 25)
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center')
plt.show()

From the graph we can see how the recession was nowhere near as significant on homicide rates in comparison to GDP growth rate. All regions remained around the same before and after it started. The two exceptions are Latin America & Caribbean as well as the Middle East & North Africa. As previously discussed, Latin America & Caribbean already had a high rate, so it increasing is not too big of a surprise. Another standout takeaway from the graph is the Middle East & North Africa experiencing a sharp decline compared to the others.

Now we know how little intentional homicide rate was influenced, we can expect there to be a low correlation between it and GDP Growth. Below is the correlation coefficient between the two variables. It is important to note that there is much more data for GDP growth rate. It spans all the way back to 1960 while intentional homicide rate only goes as far as 1995. 

In [ ]:
# Yearly global GDP growth rate average 
query = """
        SELECT i.Year, i.CountryName AS Country, AVG(i.VALUE) AS GDPGrowth
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "NY.GDP.MKTP.KD.ZG" AND c.Region != ''
        GROUP BY i.Year, i.CountryName
        """
gdp_global = pd.read_sql_query(query, conn)
gdp_global.head()


In [ ]:
# Yearly global intentional homicide rate average
query = """
        SELECT i.Year, i.CountryName AS Country, AVG(i.Value) AS HomRate
        FROM Indicators as i
        LEFT JOIN Country as c ON i.CountryCode = c.CountryCode
        WHERE i.IndicatorCode = "VC.IHR.PSRC.P5" AND c.Region != ''
        GROUP BY i.Year, i.CountryName
        """
hom_global = pd.read_sql_query(query, conn)
hom_global.head()

In [ ]:
# Merging tables by Year
global_data = pd.merge(gdp_global, hom_global, how = 'inner', on = ["Year", "Country"])
global_data.head()

In [ ]:
# Heatmap
corr = global_data[['GDPGrowth', 'HomRate']].corr()
sns.heatmap(corr, annot = True)
plt.show()

In [ ]:
# Scatter plot
plt.scatter(x = global_data['GDPGrowth'], y = global_data['HomRate'])
plt.xlabel('GDP Growth Rate')
plt.ylabel('Intentional Homicide Rate')
plt.title('Relationship Between GDP Growth and Intentional Homicide Rate')
plt.show()

As expected from earlier analysis, the correlation coefficient between the two variables is exceptionally low. However, based on the scatterplot, it may be better to use an estimate for non-linear relationships instead. The relationship appears to be more logarithmic. 

In [ ]:
# Spearman Rank Correlation Coefficient
stat = stats.spearmanr(global_data['GDPGrowth'], global_data['HomRate'])
print('Spearman Rank Correlation: ', round(stat.correlation, 3), '\nP-value: ', round(stat.pvalue, 3))

When taking into account the possibility of a non-linearity, the relationship still appears to be weak. The value is around the same, but this time it is positive instead of negative. These results imply that my initial intuition may not directly apply to homicide rate but instead to other forms of crime. 

## 5. Conclusion <a id="conclusion"></a>

From the results, we can see the majority of the 2008 recession's impact was economic. Nations worldwide were impacted greatly, especially North America as well as Western Europe. The effect was nowhere near as great in regards to crime. There was little to no change for the majority of regions. It is possible there were other crime statistics that would have shown more significant results, as there are many more types of crimes than just intentional homicide. In the future, I would like to investigate which types of crimes are most dependent on a nation's economic condition. Maybe petty crimes such as pickpocketing become more common as people start struggling more financially. Another topic that would be interesting to investigate would be how different recessions vary in impact. Comparing the 2008 recession to the recession caused by COVID-19 may yield vastly different results. 